In [1]:
import nltk
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import xlrd
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import fuzzywuzzy as fw
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
# data_xls = pd.read_excel('hate-crimes-2018.xls', 'Table 13', index_col=None)
# data_xls.to_csv('hate-crimes-2018.csv', encoding='utf-8')

In [3]:
_2018 = pd.read_csv("hate-crimes-2018.csv")

In [4]:
_2018.rename(columns={'Unnamed: 0': '1', 'Table 13': 'state', 'Unnamed: 1': 'type', 'Unnamed: 2': 'city', 'Unnamed: 3': 'race', 'Unnamed: 4': 'religion', 'Unnamed: 5': 'sexual orientation', 'Unnamed: 6': 'disability', 'Unnamed: 7': 'gender', 'Unnamed: 8': 'gender identity', 'Unnamed: 9': 'q1', 'Unnamed: 10': 'q2', 'Unnamed: 11': 'q3', 'Unnamed: 12': 'q4', 'Unnamed: 13': 'population'}, inplace=True)
_2018 = _2018.drop(columns=['1', 'q1', 'q2', 'q3', 'q4'])
_2018.state = _2018.state.fillna(method='ffill')
_2018 = _2018[pd.notnull(_2018['population'])]
_2018 = _2018.drop(columns=['type'])
_2018.head(30)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
2,Alaska,Anchorage,1.0,0.0,0.0,0.0,0.0,0.0,291992.0
3,Alaska,Fairbanks,2.0,0.0,0.0,1.0,0.0,0.0,31635.0
4,Alaska,Juneau,0.0,2.0,0.0,0.0,0.0,0.0,31922.0
5,Alaska,Kotzebue,1.0,0.0,0.0,0.0,0.0,0.0,3273.0
8,Arizona,Avondale,1.0,0.0,0.0,0.0,0.0,0.0,85204.0
9,Arizona,Coolidge,2.0,0.0,0.0,0.0,0.0,0.0,12815.0
10,Arizona,Flagstaff,1.0,0.0,1.0,0.0,0.0,0.0,72852.0
11,Arizona,Gilbert,1.0,0.0,0.0,0.0,0.0,0.0,247463.0
12,Arizona,Glendale,7.0,0.0,1.0,0.0,0.0,0.0,249799.0
13,Arizona,Maricopa,1.0,0.0,0.0,0.0,0.0,0.0,48660.0


In [14]:
_2018['city'] = _2018.city.str.strip('3')

In [6]:
_2018.sort_values(by=['race'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
131,California,Los Angeles3,136.0,63.0,74.0,0.0,0.0,10.0,4029741.0
766,Massachusetts,Boston3,89.0,18.0,39.0,0.0,0.0,2.0,694673.0
371,District of Columbia,Washington,88.0,12.0,60.0,0.0,1.0,33.0,702455.0
1379,New York,New York,84.0,216.0,41.0,0.0,0.0,10.0,8523171.0


In [7]:
_2018.sort_values(by=['religion'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
1379,New York,New York,84.0,216.0,41.0,0.0,0.0,10.0,8523171.0
131,California,Los Angeles3,136.0,63.0,74.0,0.0,0.0,10.0,4029741.0
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
17,Arizona,Phoenix3,59.0,24.0,25.0,1.0,1.0,2.0,1653080.0
766,Massachusetts,Boston3,89.0,18.0,39.0,0.0,0.0,2.0,694673.0


In [8]:
_2018.sort_values(by=['sexual orientation'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
131,California,Los Angeles3,136.0,63.0,74.0,0.0,0.0,10.0,4029741.0
371,District of Columbia,Washington,88.0,12.0,60.0,0.0,1.0,33.0,702455.0
1379,New York,New York,84.0,216.0,41.0,0.0,0.0,10.0,8523171.0
766,Massachusetts,Boston3,89.0,18.0,39.0,0.0,0.0,2.0,694673.0


In [9]:
_2018.sort_values(by=['disability'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
863,Michigan,Detroit,28.0,1.0,18.0,5.0,2.0,0.0,671275.0
1499,Ohio,Cleveland,25.0,4.0,3.0,4.0,1.0,1.0,384666.0
1501,Ohio,Columbus,42.0,6.0,19.0,4.0,0.0,0.0,873582.0
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
1522,Ohio,"Madison Township, Franklin County",0.0,0.0,0.0,3.0,0.0,0.0,19241.0


In [10]:
_2018.sort_values(by=['gender'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
890,Michigan,Highland Park,5.0,1.0,1.0,0.0,3.0,0.0,10794.0
507,Indiana,Indianapolis3,22.0,5.0,5.0,0.0,2.0,0.0,877584.0
874,Michigan,Fowlerville,0.0,0.0,0.0,0.0,2.0,0.0,2953.0
863,Michigan,Detroit,28.0,1.0,18.0,5.0,2.0,0.0,671275.0


In [11]:
_2018.sort_values(by=['gender identity'], ascending=False).head(5)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population
371,District of Columbia,Washington,88.0,12.0,60.0,0.0,1.0,33.0,702455.0
2146,Washington,Seattle,159.0,33.0,81.0,4.0,3.0,11.0,742759.0
1379,New York,New York,84.0,216.0,41.0,0.0,0.0,10.0,8523171.0
131,California,Los Angeles3,136.0,63.0,74.0,0.0,0.0,10.0,4029741.0
1639,Oregon,Eugene,18.0,7.0,9.0,1.0,0.0,4.0,170771.0


In [12]:
#     for row in _2018:
#         result = _2018['race'] + _2018['religion'] + _2018['sexual orientation'] + _2018['disability'] + _2018['gender'] + _2018['gender identity']
#         result1 = (result / _2018['population']) * 100
# _2018['percent per pop'] = result1 

In [17]:
_2018['percent per pop'] = ((_2018['race'] + _2018['religion'] + _2018['sexual orientation'] + _2018['disability'] + _2018['gender'] + _2018['gender identity']) / _2018['population']) * 100

In [20]:
_2018.sort_values(by=['percent per pop'], ascending=False)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population,percent per pop
1711,South Carolina,Elloree,4.0,0.0,0.0,0.0,0.0,0.0,652.0,0.613497
2028,Vermont,Winhall,3.0,0.0,0.0,0.0,0.0,0.0,730.0,0.410959
1727,South Carolina,Springfield,2.0,0.0,0.0,0.0,0.0,0.0,488.0,0.409836
1705,South Carolina,Calhoun Falls,5.0,0.0,0.0,1.0,0.0,0.0,1925.0,0.311688
1720,South Carolina,Lane,0.0,0.0,0.0,1.0,0.0,0.0,456.0,0.219298
903,Michigan,Mackinac Island,1.0,0.0,0.0,0.0,0.0,0.0,467.0,0.214133
2131,Washington,Morton,0.0,0.0,2.0,0.0,0.0,0.0,1164.0,0.171821
1342,New Jersey,Woodbury,12.0,0.0,3.0,1.0,0.0,0.0,9746.0,0.164170
1729,South Carolina,Winnsboro,5.0,0.0,0.0,0.0,0.0,0.0,3197.0,0.156397
1615,Oklahoma,Maud,0.0,0.0,1.0,0.0,0.0,0.0,706.0,0.141643


In [21]:
_2018.to_csv('2018-hate-crime.csv')

In [26]:
_2018[_2018.population > 10000].sort_values(by='percent per pop', ascending=False)

,state,city,race,religion,sexual orientation,disability,gender,gender identity,population,percent per pop
1927,Texas,Plainview,12.0,9.0,0.0,2.0,0.0,2.0,20569.0,0.121542
1885,Texas,Freeport,13.0,0.0,0.0,1.0,0.0,0.0,12185.0,0.114895
890,Michigan,Highland Park,5.0,1.0,1.0,0.0,3.0,0.0,10794.0,0.092644
1230,New Jersey,Hillsdale,2.0,7.0,0.0,0.0,0.0,0.0,10484.0,0.085845
1250,New Jersey,Madison,12.0,1.0,0.0,0.0,0.0,0.0,15832.0,0.082112
2213,Wisconsin,Hudson,0.0,1.0,9.0,0.0,0.0,0.0,13848.0,0.072213
1276,New Jersey,Neptune Township,8.0,8.0,2.0,0.0,0.0,1.0,27825.0,0.068284
1878,Texas,Dumas,8.0,1.0,0.0,0.0,0.0,0.0,14790.0,0.060852
1306,New Jersey,Rockaway Township,9.0,3.0,2.0,0.0,0.0,0.0,25328.0,0.055275
1349,New Jersey,New Brunswick,12.0,12.0,6.0,0.0,0.0,0.0,56027.0,0.053546


In [23]:
# data_xls = pd.read_excel('hate-crimes-2014.xls', 'Table 13', index_col=None)
# data_xls.to_csv('hate-crimes-2014.csv', encoding='utf-8')

In [ ]:
# _2014 = pd.read_csv("hate-crimes-2014.csv")

In [ ]:
# _2014.rename(columns={'Unnamed: 0': '1', 'Table 13': 'state', 'Unnamed: 1': 'type', 'Unnamed: 2': 'city', 'Unnamed: 3': 'race', 'Unnamed: 4': 'religion', 'Unnamed: 5': 'sexual orientation', 'Unnamed: 6': 'disability', 'Unnamed: 7': 'gender', 'Unnamed: 8': 'gender identity', 'Unnamed: 9': 'q1', 'Unnamed: 10': 'q2', 'Unnamed: 11': 'q3', 'Unnamed: 12': 'q4', 'Unnamed: 13': 'population'}, inplace=True)
# _2014 = _2018.drop(columns=['1', 'q1', 'q2', 'q3', 'q4'])
# _2014.state = _2018.state.fillna(method='ffill')
# _2014 = _2018[pd.notnull(_2018['population'])]
# _2014 = _2018.drop(columns=['type'])
# _2014.head(30)